In [1]:
import sys
sys.path.append('/usr/local/lib/python3.5/dist-packages/')
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import http.client, urllib.request, urllib.parse, urllib.error, base64
import json
import warnings
warnings.filterwarnings("ignore")
import pickle
import sentencepiece as spm

module_url = "https://tfhub.dev/google/universal-sentence-encoder-lite/2"
embed = hub.Module(module_url)
tf.logging.set_verbosity(tf.logging.WARN)

module = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-lite/2")
input_placeholder = tf.sparse_placeholder(tf.int64, shape=[None, None])
encodings = module(
    inputs=dict(
        values=input_placeholder.values,
        indices=input_placeholder.indices,
        dense_shape=input_placeholder.dense_shape))

with tf.Session() as sess:
    spm_path = sess.run(module(signature="spm_path"))

sp = spm.SentencePieceProcessor()
sp.Load(spm_path)
print("SentencePiece model loaded at {}.".format(spm_path))

def process_to_IDs_in_sparse_format(sp, sentences):
  # An utility method that processes sentences with the sentence piece processor
  # 'sp' and returns the results in tf.SparseTensor-similar format:
  # (values, indices, dense_shape)
    ids = [sp.EncodeAsIds(x) for x in sentences]
    max_len = max(len(x) for x in ids)
    dense_shape=(len(ids), max_len)
    values=[item for sublist in ids for item in sublist]
    indices=[[row,col] for row in range(len(ids)) for col in range(len(ids[row]))]
    return (values, indices, dense_shape)

def embed_sentence_lite(sentences):
    messages = sentences
    values, indices, dense_shape = process_to_IDs_in_sparse_format(sp, messages)

    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)

    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        message_embeddings = session.run(
          encodings,
          feed_dict={input_placeholder.values: values,
                    input_placeholder.indices: indices,
                    input_placeholder.dense_shape: dense_shape})
    
    return message_embeddings

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-lite/2'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-lite/2'.
SentencePiece model loaded at b'/tmp/tfhub_modules/539544f0a997d91c327c23285ea00c37588d92cc/assets/universal_encoder_8k_spm.model'.


In [3]:
with open('../MSR/pirate_bot/data/movie_line_pairs.txt','r') as F:
    lines = F.readlines()

In [15]:
lines[0]

'line\tresponse\n'

In [16]:
utterances = []
utter_to_response = {}
for line in lines[1:]:
    line = line.rstrip()
    p1,p2 = line.split('\t')
    utterances.append(p1)
    utter_to_response[p1] = p2

In [18]:
# len(utterances)

In [19]:
def find_closest(sentence_rep,query_rep,K):
    top_K = np.argsort(np.sqrt((np.sum(np.square(sentence_rep - query_rep),axis=1))))[:K]
    return top_K

In [11]:
import numpy as np

In [69]:
all_dataset__embeddings = np.zeros((len(utterances),512))
for i in range(0,len(utterances),10000):
    print(i)
    start = i
    end = min(i+100,len(utterances))
    chunk = utterances[start:end]
    chunk_embedding = embed_sentence_lite(chunk)
    all_dataset__embeddings[start:end] = chunk_embedding

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000


In [70]:
import pickle
with open('../MSR/pirate_bot/data/movie_line_embeddings.p','wb') as F:
    pickle.dump(all_dataset__embeddings,F)

In [71]:
# import pickle
# f = open('res/other_embeddings.p','rb')
# other_embeddings = pickle.load(f)
# f.close()

# f = open('res/your_embeddings.p','rb')
# your_embeddings = pickle.load(f)
# f.close()

# f = open('res/dilogues.p','rb')
# pr_to_sp = pickle.load(f)
# f.close()


# f = open('res/your_sents.p','rb')
# your_sentences = pickle.load(f)
# f.close()

In [72]:
# keys = list(pr_to_sp.keys())

In [73]:
# f = open('res/key_embeddings.p','rb')
# key_embeddings = pickle.load(f)
# f.close()

In [74]:
# def speak_like_me(query,K,your_embeddings,other_embeddings,your_sen):
#     other_query = [query]
#     query_embedding = embed_sentence_lite(other_query)
#     closest_your = find_closest(your_embeddings,query_embedding,K)
#     for cl in closest_your:
#         print(your_sentences[cl])

In [75]:
def respond_like_me(query,K,key_embeddings):
    other_query = [query]
    query_embedding = embed_sentence_lite(other_query)
    closest_other = find_closest(key_embeddings,query_embedding,K+2)
    return closest_other
#     for k in closest_other[3:]:
#         print([keys[k]])

In [58]:
import random

In [104]:
k = respond_like_me("ho ho ho and a bottle of rum",10,all_dataset__embeddings)

In [105]:
t = []
for i in k:
    print(utterances[i])
    t.append(utter_to_response[utterances[i]])

Let's have a go at the rum.
Hey you boys, take the bottle with you.
Here's the key to the rum store.
That's the drink.
That's what drink does to you.
Rum-looking lot, eh, sir?
We call it rum.
A bit of water!
I'm not drunk enough to go on a boat.
Land ho!
Land ho!
Water casks ready for filling, sir.


In [106]:
random.sample(t,10)

['Yeah.',
 'I noticed he was pale, take him outside!',
 "That's what drink does to you.",
 'Would that do?',
 "I'll never touch another drop of drink as long as I live.",
 'Yes, you are.',
 'I suppose I have you to thank.',
 'Where away?',
 'The ship cleared for action?',
 'You fill a pannikin and bring it back.']